In [1]:
import numpy as np
import cv2 as cv
import sys

class Sketcher:
    def __init__(self, windowname, dests, colors_func):
        self.prev_pt = None
        self.windowname = windowname
        self.dests = dests
        self.colors_func = colors_func
        self.dirty = False  #Dirty function checks if image has been dirtied (drawn over)
        self.show()
        cv.setMouseCallback(self.windowname, self.on_mouse)

    def show(self):
        cv.imshow(self.windowname, self.dests[0]) #Original Image
        cv.imshow(self.windowname+": Mask",self.dests[1]) # Masked Image
        
    def on_mouse(self, event, x, y, flags, param):
        pt = (x,y)
        if event == cv.EVENT_LBUTTONDOWN: #Left click
            self.prev_pt = pt
        elif event == cv.EVENT_LBUTTONUP:
            self.prev_pt = None
        if self.prev_pt and flags & cv.EVENT_FLAG_LBUTTON:
            for dst, color in zip(self.dests, self.colors_func()):
                cv.line(dst,self.prev_pt, pt, color, 60)
            self.dirty = True
            self.prev_pt = pt
            self.show()


def main():
    print("Usage: Python Inpaint")
    print("Keys: ")
    print("f - inpaint using FMM")
    print("n - inpaint using NS technique")
    print("r - reset the inpaint mask")
    print("ESC - exit")

    img = cv.imread("mirror_on_wall.jpeg")  #Input image to work on

    img_mask = img.copy() # Create a copy of the image

    inpainted_mask_img = np.zeros(img.shape[0:2], np.uint8) #Create a black mask image from the given input image, uint8(unsigned 8 bit integer) is the datatype of image 

    sketch = Sketcher('image',[img_mask, inpainted_mask_img], lambda :((255,255,255), 255)) # Returns the original image,masked image and the mask and lambda function defines that white line is to be drawn

    while True:
        ch = cv.waitKey(0)  # Take input from user on what to do

        if ch == 27: # ASCII code for esc key
            break
        if ch == ord('f'): #If f is pressed , FMM Method is used
            res = cv.inpaint(src=img_mask, inpaintMask=inpainted_mask_img, inpaintRadius=3, flags=cv.INPAINT_TELEA)
            cv.imshow("Inpaint output using FMM",res)
        if ch == ord('n'): # If n is pressed NS Method is used
            res = cv.inpaint(src=img_mask, inpaintMask=inpainted_mask_img, inpaintRadius=3, flags=cv.INPAINT_NS)
            cv.imshow("Inpaint output using NS",res)
        if ch == ord('r'):
            img_mask[:] = img
            inpainted_mask_img[:]=0 #Removes white lines drawn on image
            sketch.show()
    print("Completed")


if __name__ == '__main__':
    main()
    cv.destroyAllWindows()

Usage: Python Inpaint
Keys: 
f - inpaint using FMM
n - inpaint using NS technique
r - reset the inpaint mask
ESC - exit
Completed
